In [16]:
import pyJHTDB
import pyJHTDB.dbinfo
import numpy as np
import matplotlib.pyplot as plt
import time

## Train dataset
### Get velocity and pressure

In [17]:
def get_data(point_coords, time):
    """
    Get velocity and pressure at specified spatial points and a specified time in channel flow database.
    :param point_coords: Spatial coordinates of the data points of interest. Must be in single precision.
    :param time: Time of interest.
    :return: Velocity and velocity gradient arrays.
    """

    # Create library object
    lJHTDB = pyJHTDB.libJHTDB()

    # Initialize library object
    lJHTDB.initialize()

    # Get velocity
    u = lJHTDB.getData(time, point_coords,
                       sinterp='Lag4',
                       data_set='channel',
                       getFunction='getVelocity')

    # Get velocity gradient
    p = lJHTDB.getData(time, point_coords,
                            sinterp='Lag4',
                            data_set='channel',
                            getFunction='getPressure')

    # Finalize library object
    lJHTDB.finalize()

    return u, p

In [3]:
xnode = np.linspace(12.47,12.66,191)
ynode = np.linspace(-0.9,-0.7,201)
znode = np.linspace(4.61,4.82,211)
total_times = np.array(list(range(4000)), dtype = np.float32) * 0.0065
frames1 = np.arange(81)
frames2 = np.arange(129)

points = np.zeros((191, 201, 211, 3), np.float32)
points[:, :, :, 0] = xnode[:, None, None]
points[:, :, :, 1] = ynode[None, :, None]
points[:, :, :, 2] = znode[None, None, :]


In [18]:
points1 = points.reshape(-1,3)

### Initial condition (t=0)

In [53]:
idx_ini = np.random.choice(points1.shape[0], 33524, replace = False)

In [54]:
train_ini1 = points1[idx_ini, :]
train_iniv1 = np.zeros((33524, 3), np.float32)
train_inip1 = np.zeros((33524, 1), np.float32)

In [55]:
size = int(33524 / 17)
for i in range(17):
    
    start = time.time()  # start timer

    train_iniv1[0 + size * i: size + size * i, :], train_inip1[0 + size * i: size + size * i, :] = get_data(train_ini1[0 + size * i: size + size * i, :], 0)

    elapsed = time.time() - start  # end timer
    print('Elapsed time = %.2f seconds' % elapsed)

print(train_iniv1.shape, train_inip1.shape)

Elapsed time = 1.10 seconds
Elapsed time = 1.48 seconds
Elapsed time = 1.37 seconds
Elapsed time = 1.33 seconds
Elapsed time = 1.97 seconds
Elapsed time = 1.36 seconds
Elapsed time = 1.46 seconds
Elapsed time = 1.68 seconds
Elapsed time = 2.16 seconds
Elapsed time = 0.58 seconds
Elapsed time = 0.42 seconds
Elapsed time = 0.41 seconds
Elapsed time = 0.43 seconds
Elapsed time = 0.44 seconds
Elapsed time = 0.40 seconds
Elapsed time = 0.39 seconds
Elapsed time = 0.93 seconds
(33524, 3) (33524, 1)


In [56]:
np.save('train_ini1.npy',train_ini1)
np.save('train_iniv1.npy',train_iniv1)
np.save('train_inip1.npy',train_inip1)

### the boundary condition

In [57]:
points2 = points1[:, :][points1[:,0] == 12.47]
points3 = points1[:, :][points1[:,0] == 12.66]
points4 = points1[:, :][points1[:,1] == -0.9]
points5 = points1[:, :][points1[:,1] == -0.7]
points6 = points1[:, :][points1[:,2] == 4.61]
points7 = points1[:, :][points1[:,2] == 4.82]
print(points2.shape, points3.shape, points4.shape, points5.shape, points6.shape, points7.shape)

(42411, 3) (42411, 3) (40301, 3) (40301, 3) (38391, 3) (38391, 3)


In [58]:
train_b1 = np.concatenate([points2, points3, points4, points5, points6, points7],0)
train_b1.shape

(242206, 3)

In [59]:
idxb = np.random.choice(train_b1.shape[0], 6644, replace=False)

In [60]:
train_b2 = train_b1[idxb,:]
train_b2.shape

(6644, 3)

In [61]:
train_xb1 = np.zeros((6644*129, 4), np.float32)
train_vb1 = np.zeros((6644*129, 3), np.float32)
train_pb1 = np.zeros((6644*129, 1), np.float32)

In [62]:
frames = np.arange(129)
# Get data
for frame in frames:
    print(frame)
    for i in range(2):
        train_vb1[6644*frame + 3322*i: 3322+6644*frame+3322*i, :], train_pb1[6644*frame + 3322*i: 3322+6644*frame+3322*i, :] = get_data(train_b2[3322*i: 3322+3322*i, :], t)
    train_xb1[6644*frame: 6644+6644*frame, 0:3] = train_b2
    train_xb1[6644*frame: 6644+6644*frame, 3] = t

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128


In [21]:
np.save('train_xb1.npy',train_xb1)
np.save('train_vb1.npy',train_vb1)
np.save('train_pb1.npy',train_pb1)

In [63]:
x0_train = train_ini1[:,0:1]
y0_train = train_ini1[:,1:2]
z0_train = train_ini1[:,2:3]
t0_train = np.zeros(train_ini1[:,0:1].shape, np.float32)
u0_train = train_iniv1[:,0:1]
v0_train = train_iniv1[:,1:2]
w0_train = train_iniv1[:,2:3]

xb_train = train_xb1[:,0:1]
yb_train = train_xb1[:,1:2]
zb_train = train_xb1[:,2:3]
tb_train = train_xb1[:,3:4]
ub_train = train_vb1[:,0:1]
vb_train = train_vb1[:,1:2]
wb_train = train_vb1[:,2:3]

x_train1 = xnode.reshape(-1,1)[np.random.choice(191, 20000, replace=True),:]
y_train1 = ynode.reshape(-1,1)[np.random.choice(201, 20000, replace=True),:]
z_train1 = znode.reshape(-1,1)[np.random.choice(211, 20000, replace=True),:] 
x_train = np.tile(x_train1,(129, 1)) 
y_train = np.tile(y_train1,(129, 1)) 
z_train = np.tile(z_train1,(129, 1)) 

total_times1 = np.array(list(range(129))) * 0.0065 
t_train1 = total_times1.repeat(20000) 
t_train = t_train1.reshape(-1,1) 

In [65]:
print(x0_train.shape,y0_train.shape,
      z0_train.shape,t0_train.shape,
      u0_train.shape,v0_train.shape,
      w0_train.shape,
      xb_train.shape,yb_train.shape,
      zb_train.shape,tb_train.shape,
      ub_train.shape,vb_train.shape,
      wb_train.shape,
      x_train.shape, y_train.shape,
      z_train.shape, t_train.shape)

(33524, 1) (33524, 1) (33524, 1) (33524, 1) (33524, 1) (33524, 1) (33524, 1) (857076, 1) (857076, 1) (857076, 1) (857076, 1) (857076, 1) (857076, 1) (857076, 1) (2580000, 1) (2580000, 1) (2580000, 1) (2580000, 1)


## Test dataset 

In [50]:
idx_test = np.random.choice(points1.shape[0],1000,replace = False)
test_l = points1[idx_test, :] # test points locations
#choose several different time step

test_v1, test_p1 = get_data(test_l, 0.0065)
test_v2, test_p2 = get_data(test_l, 31 * 0.0065)
test_v3, test_p3 = get_data(test_l, 61 * 0.0065)
test_v4, test_p4 = get_data(test_l, 91 * 0.0065)
test_v5, test_p5 = get_data(test_l, 121 * 0.0065)

test_v = np.concatenate((test_v1, test_v2, test_v3, test_v4, test_v5), 0)
test_p = np.concatenate((test_p1, test_p2, test_p3, test_p4, test_p5), 0)

test42 = np.concatenate((test_v, test_p), 1)

np.save('test42_l.npy', test_l)
np.save('test42_vp.npy', test42)